In [102]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd 
import numpy as np
import seaborn as sns
from re import sub
import json
import requests
import time
from bs4 import BeautifulSoup as bs

sns.set_style("darkgrid")

In [103]:
%%capture
from tqdm import tqdm_notebook as tqdm

In [104]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

# Read CSVs

In [149]:
# mov_df1 = pd.read_csv("data/en_csv.csv")
mov12 = pd.read_csv("data/five_year.csv")

scrape = pd.read_csv("data/scrape.csv")


## Concatenate DataFrames

In [150]:
# mov12 = pd.concat([mov_df1, mov_df2], sort=False)

In [151]:
mov12.shape

(49978, 30)

In [152]:
scrape.drop(['Unnamed: 0'], axis = 1, inplace=True)
scrape.drop_duplicates(inplace=True)
mov12.drop_duplicates(inplace=True)

In [155]:
dfm = mov12.merge(scrape, how='left', on="imdb_id")

In [164]:
dfm.loc[dfm.budget_x == 0,'budget_x'] = dfm.loc[dfm.budget_x == 0,'budget_y']
dfm.loc[dfm.revenue_x == 0,'revenue_x'] = dfm.loc[dfm.revenue_x == 0,'revenue_y']
dfm.columns

AttributeError: 'DataFrame' object has no attribute 'budget_x'

In [157]:
cols =['Unnamed: 0', 'Unnamed: 0.1', 'adult', 'backdrop_path',
       'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'success', 'status_code',
       'status_message', 'budget_y', 'revenue_y']
dfm.columns = cols

In [158]:
dfm.dropna(subset=['genres', 'budget', 'revenue', 'vote_average'], inplace=True)
dfm.shape

(2309, 32)

In [159]:
dfm.describe()

,Unnamed: 0,Unnamed: 0.1,budget,id,popularity,revenue,runtime,vote_average,vote_count,status_code,budget_y,revenue_y
count,2309.000000,2309.000000,2.309000e+03,2309.000000,2309.000000,2.309000e+03,2276.000000,2309.000000,2309.000000,0.0,1.164000e+03,1.164000e+03
mean,2350.071460,2350.071460,9.285317e+07,459739.336942,20.748039,5.453184e+07,101.745167,5.907492,874.903421,NaN,1.536521e+08,4.997352e+06
std,2544.761509,2544.761509,1.156119e+09,110233.590836,50.718568,1.738905e+08,32.718514,2.084265,2212.912168,NaN,1.625621e+09,2.598041e+07
min,0.000000,0.000000,0.000000e+00,14564.000000,0.600000,1.000000e+00,0.000000,0.000000,0.000000,NaN,0.000000e+00,3.700000e+01
25%,280.000000,280.000000,1.000000e+06,391995.000000,2.057000,6.239800e+04,90.000000,5.500000,5.000000,NaN,7.406250e+05,2.361725e+04
50%,1293.000000,1293.000000,5.000000e+06,450045.000000,6.504000,1.598584e+06,102.000000,6.400000,40.000000,NaN,2.850000e+06,1.827230e+05
75%,3743.000000,3743.000000,2.498700e+07,525176.000000,19.619000,2.267856e+07,120.000000,7.000000,523.000000,NaN,1.012500e+07,1.490424e+06
max,9972.000000,9972.000000,3.500000e+10,761122.000000,1114.882000,2.797801e+09,227.000000,10.000000,23833.000000,NaN,3.500000e+10,4.511834e+08


In [160]:
dfm.head()

,Unnamed: 0,Unnamed: 0.1,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,...,tagline,title,video,vote_average,vote_count,success,status_code,status_message,budget_y,revenue_y
0,0,0,False,/ld7V9BjMk2xtiBNcR8savyyk5ca.jpg,NaN,110000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",http://www.foxmovies.com/movies/miss-peregrine...,283366.0,tt1935859,...,Stay peculiar,Miss Peregrine's Home for Peculiar Children,False,6.7,7533.0,NaN,NaN,NaN,NaN,NaN
2,2,2,False,/craD86vySKvAkboyeXFnZwHrNA8.jpg,"{'id': 256322, 'name': 'The Purge Collection',...",10000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",http://www.purgeelectionyear.com,316727.0,tt4094724,...,Keep America great,The Purge: Election Year,False,6.4,3735.0,NaN,NaN,NaN,NaN,NaN
3,3,3,False,/mMtUybQ6hL24FXo0F3Z4j2KG7kZ.jpg,NaN,370000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 16, ...",https://www.funimationfilms.com/movie/yourname/,372058.0,tt5311514,...,NaN,Your Name.,False,8.6,6597.0,NaN,NaN,NaN,370000000.0,358922706.0
4,4,4,False,/z9ft5HYHzWcasR6SGcgeluxTznB.jpg,"{'id': 544670, 'name': 'Sing Collection', 'pos...",75000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://www.singmovie.com/,335797.0,tt3470600,...,Auditions begin 2016.,Sing,False,7.0,5631.0,NaN,NaN,NaN,NaN,NaN
5,5,5,False,/6O0lsCK90jZCyPvyYSt8Szzlnd6.jpg,NaN,40000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",https://www.lionsgate.com/movies/hacksaw-ridge,324786.0,tt2119532,...,One of the greatest heroes in American history...,Hacksaw Ridge,False,8.1,8829.0,NaN,NaN,NaN,NaN,NaN


In [161]:
dfm = dfm.loc[dfm['budget'] > 101, :]
dfm = dfm.loc[dfm['revenue'] > 101, :]
dfm = dfm.loc[dfm['vote_average'] > .01, :]

In [162]:
dfm.shape

(2060, 32)

In [163]:
dfm['Unnamed: 0'].value_counts()

13      5
82      5
104     5
115     5
56      5
       ..
5298    1
447     1
6592    1
5073    1
6158    1
Name: Unnamed: 0, Length: 1538, dtype: int64

In [118]:
cols = ['imdb_id', 'title', 'genres', 'popularity', 'release_date', 'budget', 'revenue', 'vote_average' ]
cols_to_drop = [x for x in mov_df2.columns if x not in cols]

In [119]:
df = dfm.copy()
df.drop(columns = cols_to_drop, inplace=True)
df = df.reindex(columns=cols)
df

,imdb_id,title,genres,popularity,release_date,budget,revenue,vote_average
9,tt8850222,Peninsula,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",800.753,2020-07-15,17000000.0,35878266.0,7.0
10,tt4566758,Mulan,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",752.021,2020-09-04,200000000.0,57000000.0,7.2
13,tt4823776,The SpongeBob Movie: Sponge on the Run,"[{'id': 14, 'name': 'Fantasy'}, {'id': 16, 'na...",1114.882,2020-08-14,60000000.0,4700000.0,8.3
32,tt8461224,The Tax Collector,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",492.096,2020-08-07,30000000.0,942666.0,6.0
37,tt8060328,Invasion,"[{'id': 878, 'name': 'Science Fiction'}]",385.953,2020-01-01,8819200.0,32104054.0,7.0
...,...,...,...,...,...,...,...,...
57528,tt9239802,Qarakoz,"[{'id': 27, 'name': 'Horror'}]",1.040,2020-03-26,10000000.0,3885.0,4.0
57911,NaN,ALL COPS AIN'T BAD,"[{'id': 35, 'name': 'Comedy'}]",1.230,2020-01-01,100000.0,350000.0,10.0
59573,tt11985684,Rómulo y Julita,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",0.739,2020-02-27,200000.0,40534.0,6.0
59859,tt11579832,The Unethicals,"[{'id': 35, 'name': 'Comedy'}, {'id': 28, 'nam...",0.944,2020-01-06,129.0,129.0,10.0


In [120]:
df['ROI'] = df['revenue'] / df['budget']

In [135]:
df.release_date.sort_values()

5787     1925-07-12
4798     1927-03-06
5344     1931-02-01
7364     1931-02-12
6571     1931-11-21
            ...    
5653     2020-10-16
50848    2020-10-21
51098    2020-10-21
4967     2020-10-22
50518    2020-10-22
Name: release_date, Length: 6683, dtype: object

In [123]:
df.to_csv("data/cleaned2.csv", index= False)

In [126]:
df['budget'].value_counts()

20000000.0     229
30000000.0     207
25000000.0     205
10000000.0     203
15000000.0     184
              ... 
28200000.0       1
894000000.0      1
3328289.0        1
1950000.0        1
112.0            1
Name: budget, Length: 844, dtype: int64